Imports

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as tfrms
from sklearn.model_selection import train_test_split
import os
import cv2

Parameters

In [13]:
picture = "./train"
labelFile = "SignCSV.csv"
epochValue = 30
imageDimension = (32,32,3)
testRatio = 0.2
validationRatio = 0.2


Split Data

In [ ]:
def loadData():
    images = []
    classNo = []
    imageList = os.listdir(picture)
    print(f"Total Classes is: {len(imageList)}")

    

Data Transforms

In [8]:


# data adjustments for training
# Resiszing all of the images to 32x32

data_tfrms = tfrms.Compose([
        tfrms.Resize((32,32)),
        tfrms.ToTensor(),
        tfrms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629)),
])

# data_brightness = tfrms.Compose([
#     tfrms.Resize((32,32)),
#     tfrms.ColorJitter(brightness=-5),
#     tfrms.ColorJitter(brightness=5),
#     tfrms.ToTensor(),
#     tfrms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629)),
# ])

# data_saturation = tfrms.Compose([
#     tfrms.Resize((32,32)),
#     tfrms.ColorJitter(saturation=-5),
#     tfrms.ColorJitter(saturation=5),
#     tfrms.ToTensor(),
#     tfrms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629)),

# ])

# data_contrast = tfrms.Compose([
#     tfrms.Resize((32,32)),
#     tfrms.ColorJitter(contrast=-5),
#     tfrms.ColorJitter(contrast=5),
#     tfrms.ToTensor(),
#     tfrms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629)),

# ])

data_hue = tfrms.Compose([
    tfrms.Resize((32,32)),
    tfrms.ColorJitter(hue=0.4),
    tfrms.ToTensor(),
    tfrms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629)),

])

data_rotation = tfrms.Compose([
    tfrms.Resize((32,32)),
    tfrms.RandomRotation(15),
    tfrms.ToTensor(),
    tfrms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629)),
])

data_flips = tfrms.Compose([
    tfrms.Resize((32,32)),
    tfrms.RandomHorizontalFlip(1),
    tfrms.RandomVerticalFlip(1),
    tfrms.ToTensor(),
    tfrms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629)),
])

data_hflip = tfrms.Compose([
    tfrms.Resize((32,32)),
    tfrms.RandomHorizontalFlip(1),
    tfrms.ToTensor(),
    tfrms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629)),
])

data_vflip = tfrms.Compose([
    tfrms.Resize((32,32)),
    tfrms.RandomVerticalFlip(1),
    tfrms.ToTensor(),
    tfrms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629)),
])

data_shear = tfrms.Compose([
    tfrms.Resize((32,32)),
    tfrms.RandomAffine(degrees = 15, shear=2),
    tfrms.ToTensor(),
    tfrms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629)),
])

data_translate = tfrms.Compose([
    tfrms.Resize((32,32)),
    tfrms.RandomAffine(degrees=15, translate=(0.1,0.1)),
    tfrms.ToTensor(),
    tfrms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629)),
])

data_crop = tfrms.Compose([
    tfrms.Resize((36,36)),
    tfrms.CenterCrop(32),
    tfrms.ToTensor(),
    tfrms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629)),
])

data_grayscale = tfrms.Compose([
    tfrms.Resize((32,32)),
    tfrms.Grayscale(num_output_channels=3),
    tfrms.ToTensor(),
    tfrms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629)),
])


Convolutional Nueral Network

In [5]:
class DCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
            nn.MaxPool2d(kernel_size=2),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.MaxPool2d(kernel_size=2),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=128,out_channels=256,kernel_size=2,padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256,out_channels=320,kernel_size=3,padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=320,out_channels=256,kernel_size=3,padding=1),
            nn.MaxPool2d(kernel_size=2),
            nn.ELU(inplace=True),
        )

        self.classifcation = nn.Sequential(
            nn.Dropout(0,5),
            nn.Linear(16*256, 600),
            nn.ReLU(inplace=True),
            nn.Dropout(0,5),
            nn.Linear(in_features=600, out_features=256),
            nn.ReLU(inplace=True),
            nn.Linear(256, num_classes),
        )

    def forward(self, x):
        X = self.conv(x)
        X = X.view(X.shape[0], -1)
        Y = self.classifcation(X)
        return Y